<a href="https://colab.research.google.com/github/epochee/yolo3/blob/main/YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y tensorflow keras 

Found existing installation: Keras 2.4.2
Uninstalling Keras-2.4.2:
  Successfully uninstalled Keras-2.4.2


In [2]:
!git clone https://github.com/zzh8829/yolov3-tf2

fatal: destination path 'yolov3-tf2' already exists and is not an empty directory.


In [3]:
%cd yolov3-tf2

/content/yolov3-tf2


In [4]:
%pwd

'/content/yolov3-tf2'

In [5]:
!pip install -r requirements-gpu.txt

Obtaining file:///content/yolov3-tf2 (from -r requirements-gpu.txt (line 6))
  Attempting uninstall: yolov3-tf2
    Found existing installation: yolov3-tf2 0.1
    Can't uninstall 'yolov3-tf2'. No files were found to uninstall.
  Running setup.py develop for yolov3-tf2


In [6]:
!pip install keras==2.4.2

  Using cached Keras-2.4.2-py2.py3-none-any.whl (170 kB)


In [7]:
# 욜로 가중치 다운로드

!wget https://pjreddie.com/media/files/yolov3.weights -O data/yolov3.weights

--2022-05-12 05:28:37--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘data/yolov3.weights’

data/yolov3.weights 100%[===================>] 236.52M  62.4MB/s    in 4.0s    

2022-05-12 05:28:41 (59.0 MB/s) - ‘data/yolov3.weights’ saved [248007048/248007048]



In [8]:
#

!python convert.py --weights ./data/yolov3.weights --output ./checkpoints/yolov3.tf

2022-05-12 05:28:42.271879: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-12 05:28:43.560114: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-12 05:28:43.589762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-12 05:28:43.590374: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-12 05:28:43.590411: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-12 05:28:43.593078: I tensorflow/stream_executor/platform/default

# detect.py 참고

In [9]:
import time
import cv2

import numpy as np
import tensorflow as tf

from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs
from yolov3_tf2.models import YoloV3
from absl import app, flags, logging
#플레그의 여러가지 값들을 편리하게 지정 가능
from absl.flags import FLAGS

from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

from IPython.display import Image, display

In [10]:
flags.DEFINE_string('classes', './data/coco.names', 'classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf','weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord')
flags.DEFINE_string('output', './output.jpg', 'output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes')

In [11]:
#플레그를 재정의 했기때문에 욜로v3 초기화
app._run_init(['yolov3'], app.parse_flags_with_usage)

#재정의한 플레그들 사용
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [12]:
yolo = YoloV3(classes=FLAGS.num_classes)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


W0512 05:28:56.844016 139792163620736 deprecation.py:534] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/array_ops.py:5049: calling gather (from tensorflow.python.ops.array_ops) with validate_indices is deprecated and will be removed in a future version.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [13]:
# yolo.load_weights(FLAGS.weights).expect_partial()

# #클래스 이름을 리스트로 저장
# class_names = [c.strip() for c in open(FLAGS.classes).readlines()]


In [14]:
# img_raw = tf.image.decode_image(
#             open(FLAGS.image, 'rb').read(), channels=3)